# Divergence Measure
The main objective of this notebook is to implement the Cauchy-Schwarz Divergence Measure between two probability density functions (PDFs). To improve understanding of the code, we start by providing a walkthrough of the choices we made. 

**Table of Contents**

- [Estimating probability density functions](##Estimating-probability-density-functions)
  - [The multivariative kernel density estimator](###The-multivariative-kernel-density-estimator)
- [The Cauchy-Schawartz divergence measure](##The-Cauchy--Schawartz-divergence-measure)
  - [Aproximation 1](###Aproximation-1)
  - [Aproximation 2](###Aproximation-2)
  - [Aproximation 3](###Aproximation-3)
  - [Aproximation 4](###Aproximation-4)
- [Divergence measure function implementation](##Divergence-measure-function-implementation)
  - [Syntax](###Syntax)
  - [Input arguments](###Input-arguments)
  - [Output arguments](###Output-arguments)

## Estimating probability density functions

Let $X = [x_{1}; x_{2}; ...;x_{N}]$ be an $N-by-d$ matrix and $y = [y_{1}; y_{2}; ...;y_{N}]$ be an $N-by-1$ vector, where $x_{i}$ represents a d-dimentional observation and $y_{i} \in {1,2,...,n_{cl}}$ corresponds to the class label for $x_{i}$. 

Consider that the observations are divided into k clusters, denoted as $S_{1}$, $S_{2}$, ..., $S_{k}$. Thus S_{c} is composed of the rows of $X$ that belong to the $c$-th cluster.

Let $S$ $\in$ $[S_{1}, S_{2}, ..., S_{k}]$ be an arbitrary fixed cluster. Furthermore, given $ a < b \in [1, 2, ..., n_{cl}],$ we define:

- A = $[x_{a}; x_{a2}; ...; {{x_{a}}_{N}}_{a}]$, representing the observations from $X$ that are in cluster $S$ and have class label $a$.
- B = $[x_{b}; x_{b2}; ...; {{x_{b}}_{N}}_{b}]$, representing the observations from $X$ that are in cluster $S$ and have class label $b$.
- $V_{a}$ = $var(A)$ as a row vector containing the variance corresponding to each column of the matrix $A$. (i.e., ${[V_{a}]}_{i}$ = [${cov(A)}_{ij}]$)
- $V_{b}$ = $var(B)$ as a row vector containing the variance corresponding to each column of the matrix $B$. (i.e., ${[V_{b}]}_{i}$ = [${cov(B)}_{ij}]$)
- $p_{a}(x)$ **and** p_{b}(x) are the probability density functions (PDFs) of the process generating the d-dimensional samples in $A$ and $B$, respectively.

### The multivariative kernel density estimator
A multivariate kernel density estivator for p_{a}(x), using a Gaussian kernel function, can be expressed as: 

$\hat{p_{a}(x)} = \frac{1}{N_{a}} \sum_{i=1}^{Na} {W_{H}}_{a} (x,x_{i})$, with ${W_{H}}_{a} (x,x_{i})$ = $\frac{1}{\sqrt{(2\pi)^{d} |H_{a}|}} exp(-\frac{(x-x_{i})^{T} H_{a}^{-1} (x-x_{i})}{2})$.

Depending on the choice of $H_{a}$, a d-by-d matrix, we arrive at the distinct definitions for $\hat{p_{a}(x)}$ addressed in the literature. 

Here are the cases we will explore: 
- By considering $H_{a} = h_{a}^{2} mean(V_{a})I_{d},$ with $h_{a}^{2} = (\frac{4}{(2d + 1)N_{d}})^{\frac{2}{d+4}}$ we have the case use in [Jenssen at al. (2006) The Cauchy–Schwarz divergence and Parzen windowing: Connections to graph theory and Mercer kernels](https://www.sciencedirect.com/science/article/pii/S0016003206000767?via%3Dihub).
- By considering $H_{a} = h_{a}^{2} diag(V_{a}),$ with $h_{a}^{2} = (\frac{4}{(d + 2)N_{d}})^{\frac{2}{d+4}}$ we have the case use in [Marcelino, Pedreira (2022) Feature space partition: a local–global approach for classification](https://doi.org/10.1007/s00521-022-07647-x).
- By considering $H_a=h_a^2\,\text{cov}(A)$ or $H_a= h_a^2\,\text{mean}(V_a)\,\text{cov}(A)$, with $h_a^2 = \Big(\frac{4}{(d+2)N_a}\Big)^\frac{2}{d+4}$, we encounter the cases dicussed in [Silverman (1986) Density Estimation for Statistics and Data Analysis](https://books.google.com.br/books?id=wlQPEAAAQBAJ&lpg=PT4&ots=ionNnTnX6A&dq=978-0-412-24620-3&hl=pt-BR&pg=PA1#v=onepage&q=978-0-412-24620-3&f=false). See equation (4.7) on page 78 and additional comments on page 86 and 87.

## The Cauchy-Schawartz divergence measure
Divergence measures evaluate how close two pdfs are in some specific sense. Here we opted to use the Cauchy–Schwarz divergence, given by

$$\begin{align}\displaystyle
D_{CS}(p_a,p_b) =-\log\frac{\Big(\int p_a(\mathbf{x})p_b(\mathbf{x})d\mathbf{x}\Big)^2}{\int p_a^2(\mathbf{x})d\mathbf{x}\int p_b^2(\mathbf{x})d\mathbf{x}}\equiv- 2\log\int p_a(\mathbf{x}) p_b(\mathbf{x}) d\mathbf{x}+   \log\int p_a^2(\mathbf{x})d\mathbf{x}+  \log\int p_b^2(\mathbf{x})d\mathbf{x}.\end{align}$$

By employing the approximations $\hat{p}_a(x)$ and $\hat{p}_b(x)$ for the pdfs $p_a(x)$ and $p_b(x)$, we will calculate an estimate for the CS divergence. To begin it's important to observe that:

$$\begin{align}\displaystyle
\int \hat{p}_a(\mathbf{x}) \hat{p}_b(\mathbf{x}) d\mathbf{x} = \frac{1}{N_aN_b} \sum_{i=1}^{N_a}\sum_{j=1}^{N_b} \int W_{H_a}(\mathbf{x},\mathbf{x}_{a_i})  W_{H_b}(\mathbf{x},\mathbf{x}_{b_j})  d\mathbf{x} 
\\ 
\qquad\qquad\qquad\;\;= \frac{1}{N_aN_b} \sum_{i=1}^{N_a}\sum_{j=1}^{N_b} W_{H_a+H_b}(\mathbf{x}_{a_i},\mathbf{x}_{b_j}) 
\\ 
\qquad\qquad\qquad\;\;= \frac{1}{N_aN_b\sqrt{(2\pi)^d|H_a+H_b|}} \sum_{i=1}^{N_a}\sum_{j=1}^{N_b} \exp\Big\{ -\frac{(\mathbf{x}_{a_i}-\mathbf{x}_{b_j})(H_a+H_b)^{-1}(\mathbf{x}_{a_i}-\mathbf{x}_{b_j})\prime}{2} \Big\}.\end{align}$$

Thereby, 

$$\begin{align}\displaystyle 
\int \hat{p}_a(\mathbf{x}) \hat{p}_a(\mathbf{x}) d\mathbf{x} = \frac{1}{N_a^2\sqrt{(2\pi)^d2^d|H_a|}} \sum_{i=1}^{N_a}\sum_{j=1}^{N_a} \exp\Big\{ -\frac{(\mathbf{x}_{a_i}-\mathbf{x}_{a_j})H_a^{-1}(\mathbf{x}_{a_i}-\mathbf{x}_{a_j})\prime}{4} \Big\} 
\\ 
\qquad\qquad\qquad\;\; \equiv \frac{1}{N_a^2\sqrt{(2\pi)^d2^d|H_a|}} \Big[ N_a  + 2\sum_{j=1}^{N_a-1}\sum_{i=j+1}^{N_a} \exp\Big\{ -\frac{(\mathbf{x}_{a_i}-\mathbf{x}_{a_j})H_a^{-1}(\mathbf{x}_{a_i}-\mathbf{x}_{a_j})\prime}{4} \Big\} \Big].\end{align}$$

The above equivalence results from the matrix , defined as , being symmetric and with null diagonal.

Analogously,

$$\begin{align}\displaystyle
\int \hat{p}_b(\mathbf{x}) \hat{p}_b(\mathbf{x}) d\mathbf{x} = \frac{1}{N_b^2\sqrt{(2\pi)^d2^d|H_b|}} \Big[ N_b + 2\sum_{j=1}^{N_b-1}\sum_{i=j+1}^{N_b} \exp\Big\{ -\frac{(\mathbf{x}_{b_i}-\mathbf{x}_{b_j})H_b^{-1}(\mathbf{x}_{b_i}-\mathbf{x}_{b_j})\prime}{4} \Big\} \Big].\end{align}$$

Under such considerations, defining

$$\begin{align}\displaystyle
sum_a = N_a  + 2\sum_{j=1}^{N_a-1}\sum_{i=j+1}^{N_a} \exp\Big\{ -\frac{(\mathbf{x}_{a_i}-\mathbf{x}_{a_j})H_a^{-1}(\mathbf{x}_{a_i}-\mathbf{x}_{a_j})\prime}{4} \Big\}, \qquad sum_b = N_b  + 2\sum_{j=1}^{N_b-1}\sum_{i=j+1}^{N_b} \exp\Big\{ -\frac{(\mathbf{x}_{b_i}-\mathbf{x}_{b_j})H_b^{-1}(\mathbf{x}_{b_i}-\mathbf{x}_{b_j})\prime}{4} \Big\} \qquad\text{and}
\\
\qquad sum_{ab} = \sum_{i=1}^{N_a}\sum_{j=1}^{N_b} \exp\Big\{ -\frac{(\mathbf{x}_{a_i}-\mathbf{x}_{b_j})(H_a+H_b)^{-1}(\mathbf{x}_{a_i}-\mathbf{x}_{b_j})\prime}{2} \Big\},\end{align}$$

we have

$$\begin{align}\displaystyle
D_{CS}(\hat{p}_a,\hat{p}_b) = -\log\Big\{ \frac{N_a^2\sqrt{(2\pi)^d2^d|H_a|}  N_b^2\sqrt{(2\pi)^d2^d|H_b|}}{(N_aN_b)^2(2\pi)^d|H_a+H_b|}\; \frac{(sum_{ab})^2}{sum_a\,sum_b} \Big\} \equiv -2\log\Big\{\frac{sum_{ab}}{N_aN_b\sqrt{(2\pi)^d|H_a+H_b|}}\Big\} + \log\Big\{\frac{sum_a}{N_a^2\sqrt{(2\pi)^d2^d|H_a|}}\Big\} + \log\Big\{\frac{sum_b}{N_b^2\sqrt{(2\pi)^d2^d|H_b|}}\Big\} \\ \\ \qquad\qquad\,\,\,\equiv -\log\Big\{ \frac{2^d\sqrt{|H_a||H_b|}} {|H_a+H_b|}\; \frac{(sum_{ab})^2}{sum_a\,sum_b} \Big\} \\ \\ \qquad\qquad\,\,\,\equiv -\log\Big\{ \frac{2^d\sqrt{|H_a||H_b|}}{|H_a+H_b|} \Big\} -2\log\big\{sum_{ab}\big\} +\log\big\{sum_a\big\} +\log\big\{sum_b\big\} \\ \\ \qquad\qquad\,\,\,\equiv -d\log\big\{2\big\} -\frac{1}{2}\Big[\log\big\{|H_a|\big\}+\log\big\{|H_a|\big\}\Big] +\log\big\{|H_a+H_b|\big\} -2\log\big\{sum_{ab}\big\} +\log\big\{sum_a\big\} +\log\big\{sum_b\big\}.\end{align}$$


In the following, we will incorporate the definitions of $H_a$ and $H_b$  that we have discussed into $D_{cs}(\hat{p}_a,\hat{p}_b)$.


### Aproximation 1

By considering $H_{a} = h_{a}^{2} mean(V_{a})I_{d}$ and $H_{b} = h_{b}^{2} mean(V_{b})I_{d}$, with $h_a²$ = $h_{a}^{2} = (\frac{4}{(2d + 1)N_{d}})^{\frac{2}{d+4}}$  we have

$\displaystyle
sum_a = N_a  + 2\sum_{j=1}^{N_a-1}\sum_{i=j+1}^{N_a} \exp\Big\{ -\frac{\|\mathbf{x}_{a_i}-\mathbf{x}_{a_j}\|^2}{4h_a^2\,\text{mean}(V_a)} \Big\}, \qquad sum_b = N_b  + 2\sum_{j=1}^{N_b-1}\sum_{i=j+1}^{N_b} \exp\Big\{ -\frac{\|\mathbf{x}_{b_i}-\mathbf{x}_{b_j}\|^2}{4h_b^2\,\text{mean}(V_b)} \Big\} \qquad\text{and}\qquad sum_{ab} = \sum_{i=1}^{N_a}\sum_{j=1}^{N_b} \exp\Big\{ -\frac{\|\mathbf{x}_{a_i}-\mathbf{x}_{b_j}\|^2}{2\big(h_a^2\,\text{mean}(V_a)+h_b^2\,\text{mean}(V_b)\big)} \Big\}.$

Therefore, 

$\displaystyle
D_{CS}(\hat{p}_a,\hat{p}_b) = -d\log\big\{2\big\} -\frac{1}{2}\Big[\log\big\{\big(h_a^2\,\text{mean}(V_a)\big)^d\big\}+\log\big\{\big(h_b^2\,\text{mean}(V_b)\big)^d\big\}\Big] +\log\big\{\big(h_a^2\,\text{mean}(V_a)+h_b^2\,\text{mean}(V_b)\big)^d\big\} -2\log\big\{sum_{ab}\big\} +\log\big\{sum_a\big\} +\log\big\{sum_b\big\} \\ \\ \qquad\qquad\,\,\,\equiv -d\log\big\{2\big\} -\frac{d}{2}\Big[\log\big\{h_a^2\big\}+\log\big\{\text{mean}(V_a)\big\}+\log\big\{h_b^2\big\}+\log\big\{\text{mean}(V_b)\big\}\Big] +d\log\big\{h_a^2\,\text{mean}(V_a)+h_b^2\,\text{mean}(V_b)\big\} -2\log\big\{sum_{ab}\big\} +\log\big\{sum_a\big\} +\log\big\{sum_b\big\}.$

Bellow is the Python code for calculating $D_{CS}(\hat{p}_a,\hat{p}_b)$:

In [ ]:
Va = np.var(A, axis=0)
Vb = np.var(B, axis=0)
ha2 = ( 4/((2*d+1)*Na) )**(2/(d+4))
hb2 = ( 4/((2*d+1)*Nb) )**(2/(d+4))
logha2 = (2/(d+4))*( np.log(4) - np.log(2*d+1) - np.log(Na) )
loghb2 = (2/(d+4))*( np.log(4) - np.log(2*d+1) - np.log(Nb) )

meanVa = np.mean(Va)
meanVb = np.mean(Vb)
sum_a = np.sum(np.exp( - distance.pdist(A, 'sqeuclidean')/(4*ha2*meanVa)))
sum_b = np.sum(np.exp( - distance.pdist(B, 'sqeuclidean')/(4*hb2*meanVb)))
sum_ab = np.sum(np.exp( - distance.cdist(A,B, 'sqeuclidean')/(2*ha2*meanVa+2*hb2*meanVb)))
D_CS = -d*np.log(2) - (d/2)*( logha2 + loghb2 + np.log(meanVa) + np.log(meanVb) ) + d*np.log(ha2*meanVa+hb2*meanVb) -2*np.log(sum_ab) + np.log(Na + 2*sum_a) + np.log(Nb + 2*sum_b)


### Aproximation 2
By considering $H_{a} = h_{a}^{2} diag(V_{a})$ and $H_{b} = h_{b}^{2} diag(V_{b})$, with $h_a²$ = $(\frac{4}{(d + 2)N_{d}})^{\frac{2}{d+4}}$  we have

$\displaystyle
sum_a = N_a + 2\sum_{j=1}^{N_a-1}\sum_{i=j+1}^{N_a} \exp\Big\{ -\frac{(\mathbf{x}_{a_i}-\mathbf{x}_{a_j})  \big(\text{diag}(V_a)\big)^{-1}  (\mathbf{x}_{a_i}-\mathbf{x}_{a_j})\prime}{4h_a^2} \Big\}, \qquad sum_b = N_b + 2\sum_{j=1}^{N_b-1}\sum_{i=j+1}^{N_b} \exp\Big\{ -\frac{(\mathbf{x}_{b_i}-\mathbf{x}_{b_j})  \big(\text{diag}(V_b)\big)^{-1}  (\mathbf{x}_{b_i}-\mathbf{x}_{b_j})\prime}{4h_b^2} \Big\} \qquad\text{and}\qquad sum_{ab} = \sum_{i=1}^{N_a}\sum_{j=1}^{N_b} \exp\Big\{ -\frac{(\mathbf{x}_{a_i}-\mathbf{x}_{b_j})  \big(h_a^2\text{diag}(V_a)+h_b^2\text{diag}(V_b)\big)^{-1}  (\mathbf{x}_{a_i}-\mathbf{x}_{b_j})\prime}{2}\Big\}.$

Therefore,

$\displaystyle
D_{CS}(\hat{p}_a,\hat{p}_b)=-d\log\big\{2\big\}-\frac{1}{2}\Big[\log\big\{\big(h_a^2\big)^d\text{prod}(V_a)\big\}+\log\big\{\big(h_b^2\big)^d\text{prod}(V_b)\big\}\Big]+\log\big\{\text{prod}\big(h_a^2\,V_a+h_b^2\,V_b\big)\big\}-2\log\big\{sum_{ab}\big\}+\log\big\{sum_a\big\}+\log\big\{sum_b\big\}\\\\\qquad\qquad\,\,\,\equiv-d\log\big\{2\big\}-\frac{d}{2}\Big[ \log\big\{h_a^2\big\} + \log\big\{h_b^2\big\} \Big]-\frac{1}{2}\Big[  \log\big\{\text{prod}(V_a)\big\} + \log\big\{\text{prod}(V_b)\big\} \Big]+\log\big\{\text{prod}\big(h_a^2\,V_a+h_b^2\,V_b\big)\big\}-2\log\big\{sum_{ab}\big\}+\log\big\{sum_a\big\}+\log\big\{sum_b\big\}.
$

Bellow is the Python code for calculating $D_{CS}(\hat{p}_a,\hat{p}_b)$:

In [ ]:
Va = np.var(A, axis=0)
Vb = np.var(B, axis=0)
Va[Va<10**(-12)] = 10**(-12)
Vb[Vb<10**(-12)] = 10**(-12)
ha2 = ( 4/((d+2)*Na) )**(2/(d+4))
hb2 = ( 4/((d+2)*Nb) )**(2/(d+4))
logha2 = (2/(d+4))*( np.log(4) - np.log(d+2) - np.log(Na) )
loghb2 = (2/(d+4))*( np.log(4) - np.log(d+2) - np.log(Nb) )
logprodVa = np.sum(np.log(Va))
logprodVb = np.sum(np.log(Vb))
logprodVab = np.sum(np.log(ha2*Va+hb2*Vb))
sum_a = np.sum(np.exp( - distance.pdist(A, 'mahalanobis', VI=np.linalg.inv(np.diag(Va)))**2/(4*ha2)))
sum_b = np.sum(np.exp( - distance.pdist(B, 'mahalanobis', VI=np.linalg.inv(np.diag(Vb)))**2/(4*hb2)))
sum_ab = np.sum(np.exp( - distance.cdist(A,B, 'mahalanobis', VI=np.linalg.inv(np.diag(ha2*Va+hb2*Vb)))**2/2))

D_CS = -d*np.log(2) - (d/2)*( logha2 + loghb2 ) - (1/2)*( logprodVa + logprodVb ) + logprodVab -2*np.log(sum_ab) + np.log(Na + 2*sum_a) + np.log(Nb + 2*sum_b)


### Aproximation 3

By considering $H_a=h_a^2\,\text{cov}(A)$ and $H_b=h_b^2\,\text{cov}(B)$, with $h_a^2 = \Big(\frac{4}{(d+2)N_a}\Big)^\frac{2}{d+4}$, we have

$\displaystyle
sum_a = N_a  + 2\sum_{j=1}^{N_a-1}\sum_{i=j+1}^{N_a} \exp\Big\{ -\frac{(\mathbf{x}_{a_i}-\mathbf{x}_{a_j})  \big(\text{cov}(A)\big)^{-1}  (\mathbf{x}_{a_i}-\mathbf{x}_{a_j})\prime}{4h_a^2} \Big\}, \qquad sum_b = N_b  + 2\sum_{j=1}^{N_b-1}\sum_{i=j+1}^{N_b} \exp\Big\{ -\frac{(\mathbf{x}_{b_i}-\mathbf{x}_{b_j})  \big(\text{cov}(B)\big)^{-1}  (\mathbf{x}_{b_i}-\mathbf{x}_{b_j})\prime}{4h_b^2} \Big\} \qquad\text{and}\qquad sum_{ab} = \sum_{i=1}^{N_a}\sum_{j=1}^{N_b} \exp\Big\{ -\frac{(\mathbf{x}_{a_i}-\mathbf{x}_{b_j})  \big(h_a^2\text{cov}(A)+h_b^2\text{cov}(B)\big)^{-1}  (\mathbf{x}_{a_i}-\mathbf{x}_{b_j})\prime}{2} \Big\}.$

Therefore,

$\displaystyle
D_{CS}(\hat{p}_a,\hat{p}_b) = -d\log\big\{2\big\} -\frac{1}{2}\Big[\log\big\{\big(h_a^2\big)^d|\text{cov}(A)|\big\}+\log\big\{\big(h_b^2\big)^d|\text{cov}(B)|\big\}\Big] +\log\big\{ |h_a^2\text{cov}(A)+h_b^2\text{cov}(B)|  \big\} -2\log\big\{sum_{ab}\big\} +\log\big\{sum_a\big\} +\log\big\{sum_b\big\} 
\\ 
\\ 
\qquad\qquad\,\,\,\equiv  -d\log\big\{2\big\} -\frac{d}{2}\Big[ \log\big\{h_a^2\big\} + \log\big\{h_b^2\big\} \Big] -\frac{1}{2}\Big[  \log\big\{|\text{cov}(A)|\big\} + \log\big\{|\text{cov}(B)|\big\} \Big] +\log\big\{ |h_a^2\text{cov}(A)+h_b^2\text{cov}(B)|  \big\} -2\log\big\{sum_{ab}\big\} +\log\big\{sum_a\big\} +\log\big\{sum_b\big\}.$

Bellow is the Python code for calculating $D_{CS}(\hat{p}_a,\hat{p}_b)$:

In [ ]:
covA = np.cov(A.T)
covB = np.cov(B.T)

ha2 = ( 4/((d+2)*Na) )**(2/(d+4))
hb2 = ( 4/((d+2)*Nb) )**(2/(d+4))

logha2 = (2/(d+4))*( np.log(4) - np.log(d+2) - np.log(Na) )
loghb2 = (2/(d+4))*( np.log(4) - np.log(d+2) - np.log(Nb) )

sum_a = np.sum(np.exp( - distance.pdist(A, 'mahalanobis', VI=np.linalg.inv(covA))**2/(4*ha2)))
sum_b = np.sum(np.exp( - distance.pdist(B, 'mahalanobis', VI=np.linalg.inv(covB))**2/(4*hb2)))
sum_ab = np.sum(np.exp( - distance.cdist(A,B, 'mahalanobis', VI=np.linalg.inv(ha2*covA+hb2*covB))**2/2))
D_CS = -d*np.log(2) - (d/2)*( logha2 + loghb2 ) - (1/2)*( np.log(np.linalg.det(covA)) + np.log(np.linalg.det(covB)) ) + np.log(np.linalg.det(ha2*covA+hb2*covB)) -2*np.log(sum_ab) + np.log(Na + 2*sum_a) + np.log(Nb + 2*sum_b)


### Aproximation 4

By considering $H_a=h_a^2\,\text{mean}(V_a)\,\text{cov}(A)$ with $h_a^2 = \Big(\frac{4}{(d+2)N_a}\Big)^\frac{2}{d+4}$, we have

$\displaystyle
sum_a = N_a  + 2\sum_{j=1}^{N_a-1}\sum_{i=j+1}^{N_a} \exp\Big\{ -\frac{(\mathbf{x}_{a_i}-\mathbf{x}_{a_j})  \big(\text{cov}(A)\big)^{-1}  (\mathbf{x}_{a_i}-\mathbf{x}_{a_j})\prime}{4h_a^2\,\text{mean}(V_a)} \Big\}, \qquad sum_b = N_b  + 2\sum_{j=1}^{N_b-1}\sum_{i=j+1}^{N_b} \exp\Big\{ -\frac{(\mathbf{x}_{b_i}-\mathbf{x}_{b_j})  \big(\text{cov}(B)\big)^{-1}  (\mathbf{x}_{b_i}-\mathbf{x}_{b_j})\prime}{4h_b^2\,\text{mean}(V_b)} \Big\} \qquad\text{and}\qquad sum_{ab} = \sum_{i=1}^{N_a}\sum_{j=1}^{N_b} \exp\Big\{ -\frac{(\mathbf{x}_{a_i}-\mathbf{x}_{b_j})  \big(h_a^2\,\text{mean}(V_a)\,\text{cov}(A)+h_b^2\,\text{mean}(V_b)\,\text{cov}(B)\big)^{-1}  (\mathbf{x}_{a_i}-\mathbf{x}_{b_j})\prime}{2} \Big\}. $

Therefore,

$\displaystyle
D_{CS}(\hat{p}_a,\hat{p}_b) = -d\log\big\{2\big\} -\frac{1}{2}\Big[\log\big\{\big(h_a^2\,\text{mean}(V_a)\big)^d|\text{cov}(A)|\big\}+\log\big\{\big(h_b^2\,\text{mean}(V_b)\big)^d|\text{cov}(B)|\big\}\Big] +\log\big\{ |h_a^2\,\text{mean}(V_a)\,\text{cov}(A)+h_b^2\,\text{mean}(V_b)\,\text{cov}(B)|  \big\} -2\log\big\{sum_{ab}\big\} +\log\big\{sum_a\big\} +\log\big\{sum_b\big\} 
\\ 
\\ 
\qquad\qquad\,\,\,\equiv  -d\log\big\{2\big\} -\frac{d}{2}\Big[ \log\big\{h_a^2\big\} + \log\big\{\text{mean}(V_a)\big\} + \log\big\{h_b^2\big\} + \log\big\{\text{mean}(V_b)\big\} \Big] -\frac{1}{2}\Big[  \log\big\{|\text{cov}(A)|\big\} + \log\big\{|\text{cov}(B)|\big\} \Big] +\log\big\{ |h_a^2\,\text{mean}(V_a)\,\text{cov}(A)+h_b^2\,\text{mean}(V_b)\,\text{cov}(B)|  \big\} -2\log\big\{sum_{ab}\big\} +\log\big\{sum_a\big\} +\log\big\{sum_b\big\}.$

Bellow is the Python code for calculating $D_{CS}(\hat{p}_a,\hat{p}_b)$:

In [ ]:
Va = np.var(A, axis=0)
Vb = np.var(B, axis=0)
Va[Va<10**(-12)] = 10**(-12)
Vb[Vb<10**(-12)] = 10**(-12)

covA = np.cov(A.T)
covB = np.cov(B.T)

ha2 = ( 4/((d+2)*Na) )**(2/(d+4))
hb2 = ( 4/((d+2)*Nb) )**(2/(d+4))

logha2 = (2/(d+4))*( np.log(4) - np.log(d+2) - np.log(Na) )
loghb2 = (2/(d+4))*( np.log(4) - np.log(d+2) - np.log(Nb) )

meanVa = np.mean(Va)
meanVb = np.mean(Vb)

sum_a = np.sum(np.exp( - distance.pdist(A, 'mahalanobis', VI=np.linalg.inv(covA))**2/(4*ha2*meanVa)))
sum_b = np.sum(np.exp( - distance.pdist(B, 'mahalanobis', VI=np.linalg.inv(covB))**2/(4*hb2*meanVb)))
sum_ab = np.sum(np.exp( - distance.cdist(A,B, 'mahalanobis', VI=np.linalg.inv(ha2*meanVa*covA+hb2*meanVb*covB))**2/2))
D_CS = -d*np.log(2) - (d/2)*( logha2 + loghb2 + np.log(meanVa) + np.log(meanVb) ) - (1/2)*( np.log(np.linalg.det(covA)) + np.log(np.linalg.det(covB)) ) + np.log(np.linalg.det(ha2*meanVa*covA+hb2*meanVb*covB)) -2*np.log(sum_ab) + np.log(Na + 2*sum_a) + np.log(Nb + 2*sum_b)

## Divergence measure function implementation
Calculate the Cauchy-Schwarz Divergence Measure between two PDFs.

### Syntax
```python
D = cs_divergence_measure(A,B,DivergenceMeasure_Case)
```

### Input arguments

- $A=[\mathbf{x}_{a_1};\mathbf{x}_{a_2};\dots;\mathbf{x}_{a_{N_a}}]$ representing the observations from $X$ that are in cluster $S$ and have class label $a$;
- $B=[\mathbf{x}_{b_1};\mathbf{x}_{b_2};\dots;\mathbf{x}_{b_{N_b}}]$ representing the observations from $X$ that are in cluster $S$ and have class label $b$;
- DivergenceMeasure_case - A case number (1, 2, 3, or 4) indicating the method to use.

### Output arguments

- D_CS - The Cauchy–Schwartz divergence measure.

```python
import numpy as np
import scipy as sp
from scipy.spatial import distance
from scipy.stats import multivariate_normal

def Divergence_Measure(A,B,Divergence_Measure_Case):
    """Decription of Divergence_Measure

    Args:
        A (matrix): A represents the observations of X that are in cluster and have class label A 
        B (matrix): B represents the observations of X that are in cluster and have class label B
        Divergence_Measure_Case (integer): Divergence_Measure_Case represents the case of divergence measure that is used {0,1,2,3}

    Returns:
        int: The Cauchy-Schwarz divergence measure between A and B
    """
    D_CS = 0
    #Define Na and Nb, d 
    Na, d = A.shape
    Nb, _ = B.shape
    #switch case for the divergence measure
    if Divergence_Measure_Case == 0: 
        # x = concatenation of A and B, y = matrix of ones with dimensions (Na+Nb,1)
        x = np.concatenate((A,B), axis=0)
        y = np.concatenate((np.ones((Na,1)), np.ones((Nb,1))*2), axis=0)
        
        std1 = np.std(A, axis=0)
        std1 = std1 * ((4/((d+2)*Na))**(1/(d+4)))
        std1[std1<10**(-6)] = 10**(-6)
        std2 = np.std(B, axis=0)
        std2 = std2 * ((4/((d+2)*Nb))**(1/(d+4)))
        std2[std2<10**(-6)] = 10**(-6)

        sigma = np.concatenate((std1[:, np.newaxis], std2[:, np.newaxis]), axis=1)
        
        #finding the index for each class {1 and 2}
        index1 = np.where(y==1)[0]
        index2 = np.where(y==2)[0]

        G = np.zeros((Na+Nb,1))
        G1 = np.zeros((Na+Nb,1))
        G2 = np.zeros((Na+Nb,1))

        Y = np.ones((Na+Nb, 1)) # vector of 1's of size = n (will be useful for kronecker product)
        Y1 = np.ones((Na, 1)) # vector of 1's of size = qty of class 1 obs
        Y2 = np.ones((Nb, 1)) # vector of 1's of size = qty of class 2 obs


        
        # M1 and M2 are defined in such a way that: if M1(i)*M2(j) = 1, then the indices
        # i and j correspond to observations of different classes. This definition is
        # similar to the Information Theoretic Clustering article

        M1 = np.concatenate((np.ones((Na,1)), np.zeros((Nb,1))), axis=0).T
        M2 = np.concatenate((np.zeros((Na,1)), np.ones((Nb,1))), axis=0).T
        for i in range(0, Na+Nb): 
            #  Information potential between the 2 clusters
            #  to avoid another 'for', I do the kron prod of x(i,:) by Y. So the
            #  q would be the second for and done all at once.
            #  the product of M1(i) by M2 makes q only class diferent observations
            #  are considered
            M = np.multiply(M1[0][i], M2)
            kron = np.kron(x[i,:],Y).reshape(-1, len(x[0,:])) - x
            zer0s = np.zeros(len(x[0,:]))
            dg = np.diag(sigma[:,0]**2 + sigma[:,1]**2)
            mvpdf = multivariate_normal.pdf(kron, zer0s, dg)
            G[i] = np.sum(M * multivariate_normal.pdf(kron, zer0s, dg))
            # Information potential of the first cluster            M = M1[0][i]
            kron = np.kron(x[i,:],Y1).reshape(-1, len(x[0,:])) - x[index1,:]
            G1[i] = np.sum(M * multivariate_normal.pdf(kron, zer0s, np.diag(2*(sigma[:,0]**2))))
            # # % Information potential of the second cluster
            M = M2[0][i]
            kron = np.kron(x[i,:],Y2).reshape(-1, len(x[0,:])) - x[index2,:]
            G2[i] = np.sum(M * multivariate_normal.pdf(kron, zer0s, np.diag(2*(sigma[:,1]**2))))
            
        
        # Distance between the two clusters
        Cef = (1/(len(index1) * len(index2))) * np.sum(G)
        # Distance of the first cluster
        DCef = -2 * np.log(Cef)
        
        AuxG1 = (1/(len(index1) ** 2)) * np.sum(G1)
        DG1 = np.log(AuxG1)
        
        AuxG2 = (1/(len(index2) ** 2)) * np.sum(G2)
        DG2 = np.log(AuxG2)
        # Cauchy-Schwarz divergence measure
        D_CS = DCef + DG1 + DG2
    
    elif Divergence_Measure_Case == 1: 
        Va = np.var(A, axis=0)
        Vb = np.var(B, axis=0)
        ha2 = ( 4/((2*d+1)*Na) )**(2/(d+4))
        hb2 = ( 4/((2*d+1)*Nb) )**(2/(d+4))
        logha2 = (2/(d+4))*( np.log(4) - np.log(2*d+1) - np.log(Na) )
        loghb2 = (2/(d+4))*( np.log(4) - np.log(2*d+1) - np.log(Nb) )

        meanVa = np.mean(Va)
        meanVb = np.mean(Vb)
        sum_a = np.sum(np.exp( - distance.pdist(A, 'sqeuclidean')/(4*ha2*meanVa)))
        sum_b = np.sum(np.exp( - distance.pdist(B, 'sqeuclidean')/(4*hb2*meanVb)))
        sum_ab = np.sum(np.exp( - distance.cdist(A,B, 'sqeuclidean')/(2*ha2*meanVa+2*hb2*meanVb)))
        D_CS = -d*np.log(2) - (d/2)*( logha2 + loghb2 + np.log(meanVa) + np.log(meanVb) ) + d*np.log(ha2*meanVa+hb2*meanVb) -2*np.log(sum_ab) + np.log(Na + 2*sum_a) + np.log(Nb + 2*sum_b)
    
    elif Divergence_Measure_Case == 2: 
        Va = np.var(A, axis=0)
        Vb = np.var(B, axis=0)
        Va[Va<10**(-12)] = 10**(-12)
        Vb[Vb<10**(-12)] = 10**(-12)
        ha2 = ( 4/((d+2)*Na) )**(2/(d+4))
        hb2 = ( 4/((d+2)*Nb) )**(2/(d+4))
        logha2 = (2/(d+4))*( np.log(4) - np.log(d+2) - np.log(Na) )
        loghb2 = (2/(d+4))*( np.log(4) - np.log(d+2) - np.log(Nb) )
        logprodVa = np.sum(np.log(Va))
        logprodVb = np.sum(np.log(Vb))
        logprodVab = np.sum(np.log(ha2*Va+hb2*Vb))
        sum_a = np.sum(np.exp( - distance.pdist(A, 'mahalanobis', VI=np.linalg.inv(np.diag(Va)))**2/(4*ha2)))
        sum_b = np.sum(np.exp( - distance.pdist(B, 'mahalanobis', VI=np.linalg.inv(np.diag(Vb)))**2/(4*hb2)))
        sum_ab = np.sum(np.exp( - distance.cdist(A,B, 'mahalanobis', VI=np.linalg.inv(np.diag(ha2*Va+hb2*Vb)))**2/2))
    
        D_CS = -d*np.log(2) - (d/2)*( logha2 + loghb2 ) - (1/2)*( logprodVa + logprodVb ) + logprodVab -2*np.log(sum_ab) + np.log(Na + 2*sum_a) + np.log(Nb + 2*sum_b)

    elif Divergence_Measure_Case == 3:
        covA = np.cov(A.T)
        covB = np.cov(B.T)

        ha2 = ( 4/((d+2)*Na) )**(2/(d+4))
        hb2 = ( 4/((d+2)*Nb) )**(2/(d+4))

        logha2 = (2/(d+4))*( np.log(4) - np.log(d+2) - np.log(Na) )
        loghb2 = (2/(d+4))*( np.log(4) - np.log(d+2) - np.log(Nb) )

        sum_a = np.sum(np.exp( - distance.pdist(A, 'mahalanobis', VI=np.linalg.inv(covA))**2/(4*ha2)))
        sum_b = np.sum(np.exp( - distance.pdist(B, 'mahalanobis', VI=np.linalg.inv(covB))**2/(4*hb2)))
        sum_ab = np.sum(np.exp( - distance.cdist(A,B, 'mahalanobis', VI=np.linalg.inv(ha2*covA+hb2*covB))**2/2))
        D_CS = -d*np.log(2) - (d/2)*( logha2 + loghb2 ) - (1/2)*( np.log(np.linalg.det(covA)) + np.log(np.linalg.det(covB)) ) + np.log(np.linalg.det(ha2*covA+hb2*covB)) -2*np.log(sum_ab) + np.log(Na + 2*sum_a) + np.log(Nb + 2*sum_b)

    elif Divergence_Measure_Case == 4: 
        Va = np.var(A, axis=0)
        Vb = np.var(B, axis=0)
        Va[Va<10**(-12)] = 10**(-12)
        Vb[Vb<10**(-12)] = 10**(-12)

        covA = np.cov(A.T)
        covB = np.cov(B.T)

        ha2 = ( 4/((d+2)*Na) )**(2/(d+4))
        hb2 = ( 4/((d+2)*Nb) )**(2/(d+4))

        logha2 = (2/(d+4))*( np.log(4) - np.log(d+2) - np.log(Na) )
        loghb2 = (2/(d+4))*( np.log(4) - np.log(d+2) - np.log(Nb) )

        meanVa = np.mean(Va)
        meanVb = np.mean(Vb)

        sum_a = np.sum(np.exp( - distance.pdist(A, 'mahalanobis', VI=np.linalg.inv(covA))**2/(4*ha2*meanVa)))
        sum_b = np.sum(np.exp( - distance.pdist(B, 'mahalanobis', VI=np.linalg.inv(covB))**2/(4*hb2*meanVb)))
        sum_ab = np.sum(np.exp( - distance.cdist(A,B, 'mahalanobis', VI=np.linalg.inv(ha2*meanVa*covA+hb2*meanVb*covB))**2/2))
        D_CS = -d*np.log(2) - (d/2)*( logha2 + loghb2 + np.log(meanVa) + np.log(meanVb) ) - (1/2)*( np.log(np.linalg.det(covA)) + np.log(np.linalg.det(covB)) ) + np.log(np.linalg.det(ha2*meanVa*covA+hb2*meanVb*covB)) -2*np.log(sum_ab) + np.log(Na + 2*sum_a) + np.log(Nb + 2*sum_b)
    else :
        print('Divergence_Measure_Case not found')

    return D_CS
```
